# Homework 3

Letizia Russo, Daniel Losada Molina and Michele Luca Puzzo

Import Libraries

In [1]:
import requests
import lxml
import shutil
import funzioni as fun
import time
import nltk
import datetime
import csv
import os
import math
from tqdm import tqdm
from bs4 import BeautifulSoup
from pathlib import Path
import re
import pandas as pd
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.snowball import PorterStemmer
from nltk import stem
from nltk.tokenize import word_tokenize 
import json
import numpy
import heapq

We define variables that we will use during all the homework

In [2]:
nFolders = 299
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
url_file = 'urls.txt'
num_lines = sum(1 for line in open(url_file, encoding="utf-8")) 
pagesPerFolder = num_lines/nFolders
outputTSV = "dataset.tsv"

We create 299 folders because we have seen that we 19136 urls so we have put 64 urls in each folder

Tentative of download a single page

In [3]:
fun.downloadOneFile(1)

Cartella1.  Page_1


# 1. Data Collection

Since no dataset was provided for this homework, we had to create our own dataset by scraping the given website, which was done in the following steps.

## 1.1 Get the list of animes

Firstly, we had to get the list of all the anime we wanted to add to our dataset. This included all the anime found in the first 400 pages. Therefore, we created a function that scrapes a certain number of pages from an initial url in order to extract the url of each of them. The initial urls and the following number of pages are given as input, while the resulting urls are stored in a txt file called "urls.txt" .

In [ ]:
fun.link_list(20000) #we give in input the number of anime that we want to retrieve
#Later we will see that in the website there are less link than 20000 


## 1.2 Crawl books

In [ ]:
fun.crawl("urls.txt")

## 1.3 Parse downloaded pages

Once we got all the urls in the first 400 pages of the website, we downloaded the html corresponding to each of the collected urls and we organized them into folders. Each folder contains the htmls of the animes in page 1, page 2, ... of the list of animes.

Given all this data, it's of our interest understanding its meaning. So, in order to study it, we selected the following features: Anime Name, Anime Type, Number of episode, Release and End Dates of anime, Number of members, Score, Users, Rank, Popularity, Synopsis, Related Anime, Characters, Voices and Staff.



We assemble all these informations generating a tsv file in which every previous feature is a column.

In [ ]:
fun.tsvGenerator()

# 2. Search Engine

We import the dataset.tsv. that we have created and read it as csv to work on a pandas dataframe

In [4]:
data = pd.read_csv('dataset.tsv', sep='\t')
data.head()

,animeTitle,animeType,animeNumEpisode,releaseDate,endDate,animeNumMembers,animeScore,animeUsers,animeRank,animePopularity,animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff,Url
0,Fullmetal Alchemist: Brotherhood,TV,64.0,2009-04-05 00:00:00,2010-07-04 00:00:00,2679138,9.16,1629051,1,3,After a horrific alchemy experiment goes wrong...,"['Fullmetal Alchemist', 'Fullmetal Alchemist: ...","['Elric, Edward', 'Elric, Alphonse', 'Mustang,...","['Park, Romi', 'Kugimiya, Rie', 'Miki, Shinich...","[['Cook, Justin', 'Producer'], ['Yonai, Norito...",https://myanimelist.net/anime/5114/Fullmetal_A...
1,Gintama° (Gintama Season 4),TV,51.0,2015-04-08 00:00:00,2016-03-03 00:00:00,484546,9.09,169476,2,337,"Gintoki, Shinpachi, and Kagura return as the f...","['Gintama', 'Gintama Movie 2: Kanketsu-hen - Y...","['Sakata, Gintoki', 'Kagura', 'Shimura, Shinpa...","['Sugita, Tomokazu', 'Kugimiya, Rie', 'Sakaguc...","[['Fujita, Youichi', 'Director, Storyboard, Pl...",https://myanimelist.net/anime/28977/Gintama°
2,Shingeki no Kyojin Season 3 Part 2,TV,10.0,2019-04-02 00:00:00,2019-07-01 00:00:00,1599580,9.09,1087519,3,33,Seeking to restore humanity's diminishing hope...,"['Shingeki no Kyojin', 'Shingeki no Kyojin Sea...","['Levi', 'Yeager, Eren', 'Ackerman, Mikasa', '...","['Kamiya, Hiroshi', 'Kaji, Yuki', 'Ishikawa, Y...","[['Yabuta, Shuuhei', 'Producer'], ['Wada, Jouj...",https://myanimelist.net/anime/38524/Shingeki_n...
3,Steins;Gate,TV,24.0,2011-04-06 00:00:00,2011-09-01 00:00:00,2093244,9.09,1115839,4,11,The self-proclaimed mad scientist Rintarou Oka...,"['Steins;Gate', 'ChäoS;HEAd', 'Robotics;Notes'...","['Okabe, Rintarou', 'Makise, Kurisu', 'Shiina,...","['Miyano, Mamoru', 'Imai, Asami', 'Hanazawa, K...","[['Iwasa, Gaku', 'Producer'], ['Yasuda, Takesh...",https://myanimelist.net/anime/9253/Steins_Gate
4,Fruits Basket: The Final,TV,13.0,2021-04-06 00:00:00,2021-06-02 00:00:00,276345,9.07,113310,5,653,"Hundreds of years ago, the Chinese Zodiac spir...","['Fruits Basket', 'Fruits Basket 2nd Season', ...","['Souma, Kyou', 'Honda, Tooru', 'Souma, Yuki',...","['Uchida, Yuuma', 'Iwami, Manaka', 'Shimazaki,...","[['Ibata, Yoshihide', 'Director'], ['Aketagawa...",https://myanimelist.net/anime/42938/Fruits_Bas...


# 2.1

We use the function **cleaner** to pre-process all the the anime Description. 
In order to analyze this data, it's useful cleaning it removing stopwords, removing punctuation, doing the stemming and removing some useless words (such as 'b', 'br', 'span',  'one' ,  'id',  'none').

To maintain the original structure of the dataframe we have decided to create and worked on a copy of it

In [6]:
data2 = data.copy()
data2['animeDescription'] = data2.animeDescription.apply(lambda x: fun.cleaner(x))


We create a vocabulary in which the keys are the tokens that appear in the anime description. The values of each key are the documents in which is present that token.

In [7]:
vocabulary = fun.create_vocabulary(data2)

In [8]:
a_file = open("vocabulary.json", "w")
a_file = json.dump(vocabulary, a_file)

In [9]:
a_file = open("vocabulary.json", "r")
vocabulary = json.load(a_file)

### 2.1.1) Create your index!

At this point we were able to build the inverted index: it is a dictionary where each key is a number that correspond to a token and its values are the documents where that token appears

In [17]:
inv_lst = fun.create_inverted_list(vocabulary)

### 2.1.2) Execute the query

First of all we have to clean the query just as we have cleaned the anime description.

In [12]:
cleanQString = fun.cleaner(input("write query: "))
cleanQString

write query: SaIyan@ Race#


'saiyan race'

Then we have used the function **map_terms_id** to map the key of inverted list to token present in our vocabulary. In this way we can associate to the query's token the corresponding number. This function prints a message of error if the token is not in our list and uses the remaining token. 

In [13]:
fun.map_terms_id(vocabulary, cleanQString)

[6083, 2785]

We have noticed that for some animes there are more than one row. They are not duplicates because they have different number of members. Maybe in the website this anime is repeated in the list. Attention: they have the same description. But they have the same link, we should not have repetitions. 

In [14]:
data2[data2["animeTitle"] == "Dragon Ball Kai (Dragon Ball Z Kai)"]

,animeTitle,animeType,animeNumEpisode,releaseDate,endDate,animeNumMembers,animeScore,animeUsers,animeRank,animePopularity,animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff,Url
1036,Dragon Ball Kai (Dragon Ball Z Kai),TV,97.0,2009-04-05 00:00:00,2011-03-02 00:00:00,315593,7.71,188407,1037,542,five year event dragon ball martial art expert...,"['Dragon Ball', 'Dragon Ball Z', 'Dragon Ball ...","['Son, Gokuu', 'Vegeta', 'Son, Gohan', 'Piccol...","['Nozawa, Masako', 'Horikawa, Ryo', 'Nozawa, M...","[['Sabat, Christopher', 'Producer, ADR Directo...",https://myanimelist.net/anime/6033/Dragon_Ball...
15046,Dragon Ball Kai (Dragon Ball Z Kai),TV,97.0,2009-04-05 00:00:00,2011-03-02 00:00:00,315676,7.71,188407,1037,542,five year event dragon ball martial art expert...,"['Dragon Ball', 'Dragon Ball Z', 'Dragon Ball ...","['Son, Gokuu', 'Vegeta', 'Son, Gohan', 'Piccol...","['Nozawa, Masako', 'Horikawa, Ryo', 'Nozawa, M...","[['Sabat, Christopher', 'Producer, ADR Directo...",https://myanimelist.net/anime/6033/Dragon_Ball...
17061,Dragon Ball Kai (Dragon Ball Z Kai),TV,97.0,2009-04-05 00:00:00,2011-03-02 00:00:00,315735,7.71,188407,1037,542,five year event dragon ball martial art expert...,"['Dragon Ball', 'Dragon Ball Z', 'Dragon Ball ...","['Son, Gokuu', 'Vegeta', 'Son, Gohan', 'Piccol...","['Nozawa, Masako', 'Horikawa, Ryo', 'Nozawa, M...","[['Sabat, Christopher', 'Producer, ADR Directo...",https://myanimelist.net/anime/6033/Dragon_Ball...
19076,Dragon Ball Kai (Dragon Ball Z Kai),TV,97.0,2009-04-05 00:00:00,2011-03-02 00:00:00,315749,7.71,188407,1037,542,five year event dragon ball martial art expert...,"['Dragon Ball', 'Dragon Ball Z', 'Dragon Ball ...","['Son, Gokuu', 'Vegeta', 'Son, Gohan', 'Piccol...","['Nozawa, Masako', 'Horikawa, Ryo', 'Nozawa, M...","[['Sabat, Christopher', 'Producer, ADR Directo...",https://myanimelist.net/anime/6033/Dragon_Ball...


In the result that we obtained we see anime that in their description contain **both** the tokens of the query. We observe anime that have the same title and description: this is due to the fact that they are not duplicates because they are different in other fields like their *popularity* or *number of members*. 

In [18]:
new_dataset = fun.Search1(cleanQString, vocabulary, data2, inv_lst)
new_dataset

,animeTitle,animeDescription,Url
0,Dragon Ball Kai (Dragon Ball Z Kai),five year event dragon ball martial art expert...,https://myanimelist.net/anime/6033/Dragon_Ball...
1,Dragon Ball Kai (Dragon Ball Z Kai),five year event dragon ball martial art expert...,https://myanimelist.net/anime/6033/Dragon_Ball...
2,Dragon Ball Kai (Dragon Ball Z Kai),five year event dragon ball martial art expert...,https://myanimelist.net/anime/6033/Dragon_Ball...
3,Dragon Ball Z,five year win world martial art tournament gok...,https://myanimelist.net/anime/813/Dragon_Ball_Z
4,Dragon Ball Z,five year win world martial art tournament gok...,https://myanimelist.net/anime/813/Dragon_Ball_Z
5,Dragon Ball Z,five year win world martial art tournament gok...,https://myanimelist.net/anime/813/Dragon_Ball_Z
6,Dragon Ball Super: Broly,forti year ago planet vegeta home infam saiyan...,https://myanimelist.net/anime/36946/Dragon_Bal...
7,Dragon Ball Kai (Dragon Ball Z Kai),five year event dragon ball martial art expert...,https://myanimelist.net/anime/6033/Dragon_Ball...
8,Dragon Ball Super: Broly,forti year ago planet vegeta home infam saiyan...,https://myanimelist.net/anime/36946/Dragon_Bal...
9,Dragon Ball Super: Broly,forti year ago planet vegeta home infam saiyan...,https://myanimelist.net/anime/36946/Dragon_Bal...


Now we have dropped the animeTitle duplicates

In [19]:
new_dataset_1 = new_dataset.drop_duplicates(subset="animeTitle").reset_index(drop = True)
new_dataset_1

,animeTitle,animeDescription,Url
0,Dragon Ball Kai (Dragon Ball Z Kai),five year event dragon ball martial art expert...,https://myanimelist.net/anime/6033/Dragon_Ball...
1,Dragon Ball Z,five year win world martial art tournament gok...,https://myanimelist.net/anime/813/Dragon_Ball_Z
2,Dragon Ball Super: Broly,forti year ago planet vegeta home infam saiyan...,https://myanimelist.net/anime/36946/Dragon_Bal...
3,Dragon Ball Z Special 1: Tatta Hitori no Saish...,bardock son goku father low rank saiyan soldie...,https://myanimelist.net/anime/986/Dragon_Ball_...


# 2.2) Conjunctive query & Ranking score

Now we have created the second inverted list that has the same length of the previous one but we have added for each document the tfidf computed using the formula. 

In [25]:
inv_lst2 = fun.inverted_list_2(vocabulary, data2)

We have to obtain the query score: for each word of our query we have computed its tf-idf. 

In [20]:
query_score = fun.queries(cleanQString, vocabulary, data2)
query_score

[['saiyan', 3.246], ['race', 1.741]]

We use the function **documents** to retrieve all the pages that contain both the token of our query. So in this way it will be easier and faster to compute the cosine similarity. Indeed it will be zero for documents that do not contain these particular words, because the numerator is the dot product between the correspondent tf-idf (tf-idf of document that does not cointain all of them is zero)

In [26]:
doc = fun.documents(cleanQString, vocabulary, data2, inv_lst)
doc_score = []
for key in cleanQString.split():
    key_ = fun.map_terms_id(vocabulary, key) #each term of our query is mapped to a corresponding number
    #so in this way we can go in our inverted index. Key_ is a list of one element
    d = pd.DataFrame(inv_lst2[key_[0]], columns = ["doc", "tfidf"]) #for each token we create a df in which we have 
    #for each page that contains the single token we have tf-idf.
    word = []
    for i in doc:
        word.append(d[d["doc"] == i]["tfidf"]) #we select pages that contain both of tokens and save their tf-idf
    doc_score.append(word)
doc_score2 = list(zip(doc_score[0], doc_score[1])) #our query has two words


*doc* are the number of pages that contain both of token of the query. These pages are the solution of the first Search Engine. 

We have computed the norm two of the query: the root square of the sum of tf-idf of the query. It is always the same.

In [21]:
q_norm = fun.qnorma(query_score)
q_norm

3.6834219144702933

We compute the similarity for the documents that contain all the token in their description. 

In [27]:
num = list(range(len(doc_score2))) # initialize numerator of cosine similarity
den = list(range(len(doc_score2))) #initialize denominator of cosine similarity
anime = list(range(len(doc_score2))) #document 
similarity = list(range(len(doc_score2)))
for i in range(len(doc_score2)):
    #numerator of cosine similarity
    num[i] = query_score[0][1] * float(doc_score2[i][0]) + query_score[1][1] * float(doc_score2[i][1])
    #denominator of cosine similarity
    den[i] = math.sqrt(float(doc_score2[i][0])**2 + float(doc_score2[i][1])**2)*q_norm
    similarity[i] = num[i]/den[i]
    #link the title to the similarity obtained 
    anime[i] = data2.iloc[doc[i]]["animeTitle"]

We use a *heap data structure* to maintain the top-k documents. We have chosen k = 5. We immediately notice that the similarity  is about 1. Since the *cosine similarity* varies between -1 and 1, our result means that the words contained in the documents are mostly the same but this result doesn't imply that their order is identical.


This result was surely expectable since we have considered just two words: in fact, following the meaning of the cosine similarity, it's pretty obvious that a query composed just by two words is present in the biggest part of the documents, but if we try with a longer query, we obtain a smaller similarity.

In [28]:
top = list(zip(anime, similarity))

In [29]:
heapq.heapify(top)
heapq.nlargest(5, top)

[('Dragon Ball Z Special 1: Tatta Hitori no Saishuu Kessen',
  0.9999986609682352),
 ('Dragon Ball Z Special 1: Tatta Hitori no Saishuu Kessen',
  0.9999986609682352),
 ('Dragon Ball Z Special 1: Tatta Hitori no Saishuu Kessen',
  0.9999986609682352),
 ('Dragon Ball Z', 0.9995894082311891),
 ('Dragon Ball Z', 0.9995894082311891)]

It's the dataframe of the first Search Engine but in the last column we have put the similarity

In [30]:
new_dataset["Similarity"] = similarity
new_dataset.sort_values(by = "Similarity", ascending = False).reset_index(drop = True).head()

,animeTitle,animeDescription,Url,Similarity
0,Dragon Ball Z Special 1: Tatta Hitori no Saish...,bardock son goku father low rank saiyan soldie...,https://myanimelist.net/anime/986/Dragon_Ball_...,0.999999
1,Dragon Ball Z Special 1: Tatta Hitori no Saish...,bardock son goku father low rank saiyan soldie...,https://myanimelist.net/anime/986/Dragon_Ball_...,0.999999
2,Dragon Ball Z Special 1: Tatta Hitori no Saish...,bardock son goku father low rank saiyan soldie...,https://myanimelist.net/anime/986/Dragon_Ball_...,0.999999
3,Dragon Ball Kai (Dragon Ball Z Kai),five year event dragon ball martial art expert...,https://myanimelist.net/anime/6033/Dragon_Ball...,0.999891
4,Dragon Ball Kai (Dragon Ball Z Kai),five year event dragon ball martial art expert...,https://myanimelist.net/anime/6033/Dragon_Ball...,0.999891


Tentative of implementation with sklearn. Useless

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the tfidf and cosine similarity score for all books with the query

vectorizer = TfidfVectorizer() # Get tf-idf matrix using fit_transform function
X = vectorizer.fit_transform(data2['animeDescription'].values.astype('U')) # Store tf-idf representations of all docs

query = 'saiyan race' # Ask the user to write the query

query_vec = vectorizer.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
results = cosine_similarity(X,query_vec).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc
cos_sim = results.tolist() # Convert the cosine similarity result to list format
data2['Similarity'] = cos_sim # Write the similarity score to Similarity column of dataframe
    
#data2.nlargest(10,'Similarity') # Take the first 10 most similar books

In [ ]:
data2_top = data2.nlargest(10,'Similarity')[["animeTitle",'animeDescription', "Url", "Similarity"]].reset_index()
data2_top

# 3. Define a new score!


In [ ]:
query = input("Insert your query string: ")

In [ ]:
cleanQString = cleaner(query)
cleanQString

In [ ]:
data1 = data.copy()

In [ ]:
data1['animeTitle'] = data1.animeTitle.apply(lambda x: cleaner(x))
data1['animeDescription'] = data1.animeDescription.apply(lambda x: cleaner(x))
data1['animeRelated'] = data1.animeRelated.apply(lambda x: cleaner(x))
#data1['animeVoices'] = data1.animeVoices.apply(lambda x: cleaner(x))
#data1['animeStaff'] = data1.animeStaff.apply(lambda x: cleaner(x))

In [ ]:
data1.head()

In [ ]:
def create_vocabulary2(data):
    voc2 = {}
    for i, row in data.iterrows():
        for column in ["animeTitle", "animeDescription","animeRelated"]:
            if len(data.at[i, column]) > 0:  # check if the list is empty or not to avoid the eventually error
                token = data.at[i, column].split()
                for word in token:
                    if word in voc2.keys(): # insert the token into the vocabulary with the documents where this is present
                        if i not in voc2[word]:
                            voc2[word].append(i)
                    else:
                        voc2[word] = [i]
    return voc2

In [ ]:
def inverted_list_3(voc2, df):

    inv_lst3 = {}

    indexes = list(voc2.keys())
    for key in voc2.keys():
        lst_doc = voc2[key]

        result = []

        for doc in lst_doc: # extract the list of tokens from a proper column
            tf_idf = []
            for column in ["animeTitle", "animeDescription","animeRelated"]: # insert all tokens present in those columns
                interested_row = df.at[doc, column]
            #interested_word = key #i-th word
                try:
                    tf = interested_row.count(key) / len(interested_row)
                except:
                    tf = 0
                idf = math.log(len(df)/len(lst_doc))
                
                tf_idf.append(round(tf * idf, 3))

            result.append((doc, sum(tf_idf)/3))

        inv_lst3[indexes.index(key)] = result # insert the result into the inverted list

    return inv_lst3

In [ ]:
vocabulary2 = create_vocabulary2(data1)
import math

In [ ]:
b_file = open("vocabulary2.json", "w")
b_file = json.dump(vocabulary2, b_file)
b_file = open("vocabulary2.json", "r")
vocabulary2 = json.load(b_file)

In [ ]:
inv_lst3 = inverted_list_3(vocabulary2, data1)

# 5. Algorithmic question

The algorithm is using Dynamic Programing to solve it. Basically, we use 2 variables to know if we should take the i-th value or not. These variables are, incl, which will store the higher sum we can get including the i-th value, and excl, which will store the higher sum we can get without including the i-th value. We start iterating the list of values from the first index. So initially incl = or[0], Because the higher sum we can obtain till the moment it's getting the first element. And excl = 0, because as we are in the beginning, if we don't take the only element we have, the sum is 0. Hereinafter, for each of the next documents, we will check if the best sum we got excluding the last element plus the current element, it's better or not than the highest sum including the last element. If the sum of including the last element its higher, we can't use the current one because are consecutive. If the sum of the excl and the current is higher, means that till the moment we are going to include in our solution the current element. If we keep going , at the end of the array we will obtain the highest sum of the whole array.

To also get the values that are used in the correct solution, we can store in an array the values that we are including. The problem is that as we can't have consecutive elements, if we decide that we want to use the current value, and the last value iterated is in the solution array, we have to switch to the state of the solution before we added the last element, and append the current one. In case that the last element is not in the list, we don't have any conflict into adding the current one. That way, at every iteration, we have the values used for getting the best solution till the moment, so at the end of the array, we will also have the values that we used for the solution.

In [31]:
def optimalSol(arr):
    incl = arr[0] # list of max sum including the previous element 
    excl = 0 #list of max sum excluding the previous element
    indexOfOptimalSol = [0] #index of the best solution 
    #print("$$$$$$$$$$$$$$$$$$$$")
    #print("index: ", 0)
    #print("incl = ", incl)
    #print("excl = ", excl)
    #print("indexOfOptimalSol: ", indexOfOptimalSol)
    #print("$$$$$$$$$$$$$$$$$$$$")
    lastElem = 0
    lastIndexArrayState = []
    for i in range(1,len(arr)): # i is the actual position
        oldIncl = incl
        if excl+arr[i] > incl:
            aux = indexOfOptimalSol
            if i - 1 in indexOfOptimalSol:
                indexOfOptimalSol = lastIndexArrayState + [i] #update of the index in the best solution
            else: 
                indexOfOptimalSol = indexOfOptimalSol + [i] #update of the index in the best solution
            lastIndexArrayState = aux
            incl = excl+arr[i]
        excl = oldIncl
        #print("$$$$$$$$$$$$$$$$$$$$")
        #print("index: ", i)
        #print("incl = ", incl)
        #print("excl = ", excl)
        #print("indexOfOptimalSol: ", indexOfOptimalSol)
        #print("$$$$$$$$$$$$$$$$$$$$")
        lastElem = i
        
        
    
    sol = list(map(lambda x: arr[x],indexOfOptimalSol))
    print("SOLUTION: ", incl)
    print("The solution is the sum of the elements: ", sol)
    return incl, sol
    

example

In [32]:
optimalSol([30, 40, 25, 50, 30, 20])

SOLUTION:  110
The solution is the sum of the elements:  [40, 50, 20]


(110, [40, 50, 20])

Program that given in input an instance in the form given above, gives the optimal solution.


In [34]:
print("Good morning personal trainer!")
n = int(input("How many appointments do you have today? ")) #len of the list
l = []
for app in range(1, n+1):
    l.append(int(input("How much time want to spend the "+ str(app)+"th client? "))) #item of the list
optimalSol(l)

Good morning personal trainer!
How many appointments do you have today? 5
How much time want to spend the 1th client? 60
How much time want to spend the 2th client? 30
How much time want to spend the 3th client? 20
How much time want to spend the 4th client? 50
How much time want to spend the 5th client? 45
SOLUTION:  125
The solution is the sum of the elements:  [60, 20, 45]


(125, [60, 20, 45])